# Problem 1
● Current Stock Price $151.03  
● Strike Price $165  
● Current Date 03/13/2022  
● Options Expiration Date 04/15/2022  
● Risk Free Rate of 4.25%  
● Continuously Compounding Coupon of 0.53%  
Implement the closed form greeks for GBSM.    
Implement a finite difference derivative calculation.  
Compare the values between the two methods for both a call and a put.  
Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above:  
● Pays dividend on 4/11/2022 of $0.88  
Calculate the value of the call and the put. Calculate the Greeks of each.  
What is the sensitivity of the put and call to a change in the dividend amount?

In [28]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [38]:
def binomial_tree_american_option(S, K, T, r, q, sigma, steps, option_type="call", dividend_payment=None, dividend_date=None):
    """
    使用二叉树估值美式期权价格（包括分红调整）
    参数：
        S: 当前股价
        K: 行权价
        T: 到期时间（年）
        r: 无风险利率
        q: 连续复利股息率
        sigma: 波动率
        steps: 二叉树步数
        option_type: "call" 或 "put" 表示看涨或看跌期权
        dividend_payment: 分红金额（如果有）
        dividend_date: 分红日期（以年为单位），即从当前日期开始的时间
    返回：
        期权价格
    """
    dt = T / steps  # 每步的时间间隔
    u = np.exp(sigma * np.sqrt(dt))  # 上升比例
    d = 1 / u  # 下降比例
    p = (np.exp((r - q) * dt) - d) / (u - d)  # 风险中性概率

    # 初始化股票价格树
    stock_tree = np.zeros((steps + 1, steps + 1))
    for i in range(steps + 1):
        for j in range(i + 1):
            stock_tree[j, i] = S * (u ** (i - j)) * (d ** j)
            # 如果设置了分红，在分红日期调整股票价格
            if dividend_payment is not None and dividend_date is not None:
                if i * dt >= dividend_date:
                    stock_tree[j, i] -= dividend_payment

    # 初始化期权价值树
    option_tree = np.zeros((steps + 1, steps + 1))
    for j in range(steps + 1):
        if option_type == "call":
            option_tree[j, steps] = max(0, stock_tree[j, steps] - K)  # 看涨期权的到期价值
        elif option_type == "put":
            option_tree[j, steps] = max(0, K - stock_tree[j, steps])  # 看跌期权的到期价值

    # 反向迭代计算期权价值
    for i in range(steps - 1, -1, -1):
        for j in range(i + 1):
            # 折现后的期权持有价值
            option_value_if_held = np.exp(-r * dt) * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])
            # 美式期权可以提前行权，取持有价值和立即行权价值的较大值
            if option_type == "call":
                option_tree[j, i] = max(option_value_if_held, stock_tree[j, i] - K)
            elif option_type == "put":
                option_tree[j, i] = max(option_value_if_held, K - stock_tree[j, i])

    return option_tree[0, 0]

# 参数设置
S = 151.03  # 当前股价
K = 165  # 行权价
T = (30 + 15 + 1) / 365  # 从3月13日到4月15日的到期时间（年）
r = 0.0425  # 无风险利率
q = 0.0053  # 年化股息率（假设为连续复利）
sigma = 0.20  # 隐含波动率
steps = 1000  # 二叉树步数，越高精度越高

# 计算无分红情况下的美式看涨和看跌期权价格
call_price_no_dividend = binomial_tree_american_option(S, K, T, r, q, sigma, steps, "call")
put_price_no_dividend = binomial_tree_american_option(S, K, T, r, q, sigma, steps, "put")

# 计算有分红情况下的美式看涨和看跌期权价格
dividend_payment = 0.88  # 每股分红
dividend_date = (30 + 11) / 365  # 分红日期为4月11日，折算成年
call_price_with_dividend = binomial_tree_american_option(S, K, T, r, q, sigma, steps, "call", dividend_payment, dividend_date)
put_price_with_dividend = binomial_tree_american_option(S, K, T, r, q, sigma, steps, "put", dividend_payment, dividend_date)

# 输出结果
print(f"American call option price without dividend: {call_price_no_dividend:.2f}")
print(f"American put option price without dividend: {put_price_no_dividend:.2f}")
print(f"American call option price with dividend: {call_price_with_dividend:.2f}")
print(f"American put option price with dividend: {put_price_with_dividend:.2f}")

American call option price without dividend: 0.65
American put option price without dividend: 14.18
American call option price with dividend: 0.60
American put option price with dividend: 14.69


股息的支付降低了美式看涨期权的价值，因为持有看涨期权的投资者会在股息支付前倾向于提前行权，以获取股息，这减少了期权的时间价值。相反，股息的存在略微增加了看跌期权的价值，因为股息支付会导致标的资产价格下降，从而提高了看跌期权行权的可能性。  
The payment of dividends reduces the value of an American call option because investors holding a call option will tend to exercise the option early to capture the dividend before it is paid, which reduces the time value of the option. Conversely, the presence of a dividend slightly increases the value of a put option because the dividend payment causes the price of the underlying asset to fall, which increases the likelihood that the put option will be exercised.

In [39]:
# 计算Greeks
def calculate_greeks(S, K, T, r, q, sigma):
    """
    计算Black-Scholes模型的Greeks：Delta, Gamma, Vega, Theta, Rho
    """
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Delta
    delta_call = np.exp(-q * T) * norm.cdf(d1)
    delta_put = np.exp(-q * T) * (norm.cdf(d1) - 1)
    
    # Gamma
    gamma = (np.exp(-q * T) * norm.pdf(d1)) / (S * sigma * np.sqrt(T))
    
    # Vega
    vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
    
    # Theta
    theta_call = (-S * norm.pdf(d1) * sigma * np.exp(-q * T) / (2 * np.sqrt(T)) -
                  r * K * np.exp(-r * T) * norm.cdf(d2) +
                  q * S * np.exp(-q * T) * norm.cdf(d1))
    
    theta_put = (-S * norm.pdf(d1) * sigma * np.exp(-q * T) / (2 * np.sqrt(T)) +
                 r * K * np.exp(-r * T) * norm.cdf(-d2) -
                 q * S * np.exp(-q * T) * norm.cdf(-d1))
    
    # Rho
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2)
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return {
        "Delta": {"call": delta_call, "put": delta_put},
        "Gamma": gamma,
        "Vega": vega,
        "Theta": {"call": theta_call, "put": theta_put},
        "Rho": {"call": rho_call, "put": rho_put}
    }

# 计算并显示Greeks
greeks = calculate_greeks(S, K, T, r, q, sigma)
print("Greeks：")
print(f"Delta (call): {greeks['Delta']['call']:.4f}")
print(f"Delta (put): {greeks['Delta']['put']:.4f}")
print(f"Gamma: {greeks['Gamma']:.4f}")
print(f"Vega: {greeks['Vega']:.4f}")
print(f"Theta (call): {greeks['Theta']['call']:.4f}")
print(f"Theta (put): {greeks['Theta']['put']:.4f}")
print(f"Rho (call): {greeks['Rho']['call']:.4f}")
print(f"Rho (put): {greeks['Rho']['put']:.4f}")


Greeks：
Delta (call): 0.1261
Delta (put): -0.8732
Gamma: 0.0193
Vega: 11.1044
Theta (call): -9.4920
Theta (put): -3.3168
Rho (call): 2.3185
Rho (put): -18.3650


In [46]:
import numpy as np
from scipy.stats import norm

# Black-Scholes 期权定价函数
def black_scholes_call_put(S, K, T, r, q, sigma, option_type="call"):
    """
    Black-Scholes 公式，用于计算欧式看涨/看跌期权的价格
    """
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == "call":
        price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type")
    return price

# 计算有限差分法 Greeks，包括 Delta, Gamma, Vega, Theta, Rho
def finite_difference_greeks(S, K, T, r, q, sigma, option_type="call", epsilon=1e-4):
    """
    使用有限差分法近似Greeks：Delta, Gamma, Vega, Theta, Rho
    """
    # 原始期权价格
    option_price = black_scholes_call_put(S, K, T, r, q, sigma, option_type)
    
    # Delta
    price_up = black_scholes_call_put(S + epsilon, K, T, r, q, sigma, option_type)
    price_down = black_scholes_call_put(S - epsilon, K, T, r, q, sigma, option_type)
    delta_fd = (price_up - price_down) / (2 * epsilon)
    
    # Gamma
    gamma_fd = (price_up - 2 * option_price + price_down) / (epsilon ** 2)
    
    # Vega
    price_vega = black_scholes_call_put(S, K, T, r, q, sigma + epsilon, option_type)
    vega_fd = (price_vega - option_price) / epsilon
    
    # Theta (时间变化对价格的敏感性)
    price_theta = black_scholes_call_put(S, K, T - epsilon, r, q, sigma, option_type)
    theta_fd = (price_theta - option_price) / epsilon
    
    # Rho (利率变化对价格的敏感性)
    price_rho = black_scholes_call_put(S, K, T, r + epsilon, q, sigma, option_type)
    rho_fd = (price_rho - option_price) / epsilon
    
    return {
        "Delta (FD)": delta_fd, 
        "Gamma (FD)": gamma_fd, 
        "Vega (FD)": vega_fd, 
        "Theta (FD)": theta_fd, 
        "Rho (FD)": rho_fd
    }

# finite difference derivative calculation Greeks（call）
fd_greeks_call = finite_difference_greeks(S, K, T, r, q, sigma, option_type="call")
print("Greeks（call option）：")
print(f"Delta (FD): {fd_greeks_call['Delta (FD)']:.4f}")
print(f"Gamma (FD): {fd_greeks_call['Gamma (FD)']:.4f}")
print(f"Vega (FD): {fd_greeks_call['Vega (FD)']:.4f}")
print(f"Theta (FD): {fd_greeks_call['Theta (FD)']:.4f}")
print(f"Rho (FD): {fd_greeks_call['Rho (FD)']:.4f}")

# finite difference derivative calculation Greeks（Put）
fd_greeks_put = finite_difference_greeks(S, K, T, r, q, sigma, option_type="put")
print("\nGreeks（put option）：")
print(f"Delta (FD): {fd_greeks_put['Delta (FD)']:.4f}")
print(f"Gamma (FD): {fd_greeks_put['Gamma (FD)']:.4f}")
print(f"Vega (FD): {fd_greeks_put['Vega (FD)']:.4f}")
print(f"Theta (FD): {fd_greeks_put['Theta (FD)']:.4f}")
print(f"Rho (FD): {fd_greeks_put['Rho (FD)']:.4f}")

Greeks（call option）：
Delta (FD): 0.1261
Gamma (FD): 0.0193
Vega (FD): 11.1082
Theta (FD): -9.4907
Rho (FD): 2.3188

Greeks（put option）：
Delta (FD): -0.8732
Gamma (FD): 0.0193
Vega (FD): 11.1082
Theta (FD): -3.3156
Rho (FD): -18.3645


两种方法的计算结果非常接近，这说明在当前的输入条件下，两种方法计算的希腊字母值均可靠。这两种方法的结果一致性验证了计算的准确性。  
The results of the two methods are very close to each other, which indicates that the Greek letter values calculated by both methods are reliable under the current input conditions. The consistency of the results of these two methods verifies the accuracy of the calculations.

# Problem 2
Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and assuming :  
● American Options  
● Current Date 03/03/2023  
● Current AAPL price is 165  
● Risk Free Rate of 4.25%  
● Dividend Payment of 1.00 on 3/15/2023  

Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above). Calculate Mean, VaR and ES.  
Calculate VaR and ES using Delta-Normal.  
Present all VaR and ES values a $ loss, not percentages.  
Compare these results to last week’s results.

In [167]:
from scipy.stats import norm
from scipy.optimize import root_scalar
import numpy as np

# GBSM函数
def gbsm(call, S, X, ttm, rf, b, iv):
    d1 = (np.log(S / X) + (b + iv**2 / 2) * ttm) / (iv * np.sqrt(ttm))
    d2 = d1 - iv * np.sqrt(ttm)
    if call:
        value = S * np.exp((b - rf) * ttm) * norm.cdf(d1) - X * np.exp(-rf * ttm) * norm.cdf(d2)
    else:
        value = X * np.exp(-rf * ttm) * norm.cdf(-d2) - S * np.exp((b - rf) * ttm) * norm.cdf(-d1)
    return value

# 定义隐含波动率的求解函数
def f(iv):
    return gbsm(True, S, X, ttm, rf, rf, iv) - p

# 输入参数
S = 100.50  # 标的物价格
X = 100     # 行权价
ttm = 15 / 255  # 剩余期限，按交易日计算
rf = 0.0025  # 无风险利率
p = 2.5      # 市场价格

# 检查区间端点值
print("f(0.001) =", f(0.001))
print("f(2.0) =", f(2.0))

# 如果端点值符号相同，尝试扩大区间或检查输入参数
try:
    result = root_scalar(f, bracket=[0.001, 2.0], method='bisect')
    if result.converged:
        implied_vol = result.root
        print(f"implied volatility: {implied_vol:.4f}")
    else:
        print("隐含波动率求解未收敛，请检查输入参数。")
except ValueError as e:
    print("错误:", e)
    print("尝试扩大区间或调整初始参数。")

f(0.001) = -1.9852951989089291
f(2.0) = 16.96839180948028
implied volatility: 0.2303


In [151]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# 加载 AAPL 股票价格数据
aapl_prices = daily_prices[['Date', 'AAPL']].dropna()
aapl_prices['Date'] = pd.to_datetime(aapl_prices['Date'])
aapl_prices.set_index('Date', inplace=True)

# 删除 OptionType 列中为空或无效的行，只保留 "Call" 和 "Put" 类型
problem2 = problem2.dropna(subset=['OptionType'])
problem2 = problem2[problem2['OptionType'].isin(['Call', 'Put'])]


In [168]:
# 模拟设置
nSim = 5000  # 模拟次数
days_ahead = 10
current_price = 165  # 给定当前 AAPL 价格（2023 年 3 月 3 日）

# 生成 10 天的模拟收益率，假设均值为 0，波动率为 aapl_std_dev
simulated_returns = np.random.normal(0, implied_vol, (nSim, days_ahead))

# 将累计收益率应用到当前价格上，得到未来的模拟价格
simulated_prices = current_price * np.exp(np.sum(simulated_returns, axis=1))

# 检查模拟的 AAPL 价格范围
print(f"\nSimulated AAPL price range: min = {simulated_prices.min():.2f}, max = {simulated_prices.max():.2f}")


Simulated AAPL price range: min = 8.26, max = 2113.05


In [153]:
# 定义函数来计算每个期权的损益
def calculate_option_pnl(option_type, strike, option_current_price, holding, simulated_prices):
    """
    计算每个期权的损益。
    option_type: 期权类型（"Call" 或 "Put"）
    strike: 行权价
    option_current_price: 当前期权价格
    holding: 持仓数量（正数为多头，负数为空头）
    simulated_prices: 模拟的标的价格
    """
    if option_type == 'Call':
        intrinsic_value = np.maximum(simulated_prices - strike, 0)
    elif option_type == 'Put':
        intrinsic_value = np.maximum(strike - simulated_prices, 0)
    else:
        raise ValueError("Invalid option type")
    
    # 返回持仓的损益
    return holding * (intrinsic_value - option_current_price)

# 初始化未对冲的总 PnL 数组
pnl_total = np.zeros(nSim)

# 初始化每个组合的 PnL 统计字典
portfolio_metrics = {}

# 遍历期权组合，计算每个组合的 PnL，并按组合名称累加
for index, row in problem2.iterrows():
    portfolio_name = row['Portfolio']
    option_type = row['OptionType']
    strike = row['Strike']
    holding = row['Holding']
    option_current_price = row['CurrentPrice']
    
    # 计算每个期权的 PnL
    pnl_option = calculate_option_pnl(option_type, strike, option_current_price, holding, simulated_prices)
    pnl_total += pnl_option  # 加入到总 PnL
    
    # 初始化组合的 PnL 数组
    if portfolio_name not in portfolio_metrics:
        portfolio_metrics[portfolio_name] = pnl_option
    else:
        portfolio_metrics[portfolio_name] += pnl_option  # 累加组合的 PnL

# 计算每个组合的均值、VaR 和 ES
individual_metrics = {}
for portfolio_name, pnl in portfolio_metrics.items():
    mean_individual = pnl.mean()
    VaR_individual = np.percentile(pnl, 5)  # 5% VaR
    ES_individual = pnl[pnl <= VaR_individual].mean()  # 预期损失 ES
    
    # 存储每个组合的结果
    individual_metrics[portfolio_name] = {
        "Mean": mean_individual,
        "VaR": abs(VaR_individual),
        "ES": abs(ES_individual)
    }

# 计算未对冲的总组合的均值、VaR 和 ES
mean_unhedged = pnl_total.mean()
VaR_unhedged = np.percentile(pnl_total, 5)  # 5% VaR
ES_unhedged = pnl_total[pnl_total <= VaR_unhedged].mean()  # 预期损失 ES

# 输出总组合的结果
print(f"Total Portfolio - Mean: ${mean_unhedged:.2f}")
print(f"Total Portfolio - VaR: ${abs(VaR_unhedged):.2f}")
print(f"Total Portfolio - ES: ${abs(ES_unhedged):.2f}")

# 输出每个单独组合的结果
for portfolio_name, metrics in individual_metrics.items():
    print(f"Portfolio {portfolio_name} - Mean: ${metrics['Mean']:.2f}, VaR: ${metrics['VaR']:.2f}, ES: ${metrics['ES']:.2f}")


Total Portfolio - Mean: $13.24
Total Portfolio - VaR: $8.45
Total Portfolio - ES: $7.28
Portfolio Straddle - Mean: $3.37, VaR: $0.98, ES: $0.39
Portfolio SynLong - Mean: $13.07, VaR: $10.68, ES: $10.09
Portfolio CallSpread - Mean: $5.41, VaR: $5.41, ES: $5.41
Portfolio PutSpread - Mean: $-3.01, VaR: $3.01, ES: $3.01
Portfolio Call  - Mean: $8.22, VaR: $5.83, ES: $5.24
Portfolio Put  - Mean: $-4.85, VaR: $4.85, ES: $4.85
Portfolio CoveredCall - Mean: $-5.97, VaR: $8.42, ES: $9.10
Portfolio ProtectedPut - Mean: $-3.01, VaR: $3.01, ES: $3.01


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# 定义计算 Delta 的函数（基于 Black-Scholes 公式）
def calculate_delta(S, K, T, r, sigma, option_type="call"):
    """
    使用 Black-Scholes 公式计算欧式期权的 Delta
    S: 标的资产价格
    K: 行权价
    T: 到期时间（以年为单位）
    r: 无风险利率
    sigma: 波动率
    option_type: 期权类型（"Call" 或 "Put"）
    """
    option_type = option_type.strip()  
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if option_type.lower() == "call":
        return norm.cdf(d1)  # 看涨期权的 Delta
    elif option_type.lower() == "put":
        return -norm.cdf(-d1)  # 看跌期权的 Delta
    else:
        raise ValueError(f"Invalid option type: {option_type}")

# 计算动态对冲后的 PnL
def calculate_dynamic_hedged_pnl(problem2_filtered, simulated_returns, days_ahead, current_price, aapl_std_dev, rf):
    """
    通过动态调整组合中的每个期权 Delta 来计算对冲后的 PnL
    """
    nSim = simulated_returns.shape[0]
    pnl_dynamic_hedged = np.zeros(nSim)
    portfolio_pnl = {name: np.zeros(nSim) for name in problem2_filtered['Portfolio'].unique()}
    
    # 在每一天根据组合的 Delta 动态调整对冲
    for day in range(days_ahead):
        net_delta = 0  # 组合的净 Delta
        for index, row in problem2_filtered.iterrows():
            option_type = row['OptionType']
            strike = row['Strike']
            holding = row['Holding']
            expiration_days = (pd.to_datetime(row['ExpirationDate']) - pd.Timestamp("2023-03-03")).days / 365
            portfolio_name = row['Portfolio']  # 获取组合名称
            
            # 计算每一天的剩余到期时间
            time_remaining = expiration_days - day / 255
            # 计算每个期权的 Delta 并累加到净 Delta
            option_delta = calculate_delta(current_price, strike, time_remaining, rf, aapl_std_dev, option_type)
            net_delta += holding * option_delta
        
            # 计算每个期权的 PnL 调整
            hedge_adjustment_day = holding * option_delta * current_price * simulated_returns[:, day]
            portfolio_pnl[portfolio_name] += hedge_adjustment_day
        
        # 使用组合的净 Delta 对模拟收益进行调整
        hedge_adjustment_day = net_delta * current_price * simulated_returns[:, day]
        pnl_dynamic_hedged += hedge_adjustment_day

    # 合并未对冲的 PnL 与动态对冲调整
    pnl_hedged_final = pnl_total + pnl_dynamic_hedged
    
    # 将对冲后的总 PnL 与每个组合的 PnL 返回
    return pnl_hedged_final, portfolio_pnl

# 设置参数
days_ahead = 10
current_price = 165  # 当前 AAPL 价格
aapl_std_dev = 0.2  # 假设波动率（可以根据历史数据计算）
rf = 0.0425  # 无风险利率
nSim = 10000  # 模拟次数
simulated_returns = np.random.normal(0, aapl_std_dev / np.sqrt(255), (nSim, days_ahead))  # 每日模拟收益

# 初始化未对冲的总 PnL 数组
pnl_total = np.zeros(nSim)

# 计算对冲后的 PnL
pnl_hedged_final, portfolio_pnl = calculate_dynamic_hedged_pnl(problem2, simulated_returns, days_ahead, current_price, aapl_std_dev, rf)

# 计算对冲后的总组合的均值、VaR 和 ES
mean_hedged_dynamic = pnl_hedged_final.mean()
VaR_hedged_dynamic = np.percentile(pnl_hedged_final, 5)
ES_hedged_dynamic = pnl_hedged_final[pnl_hedged_final <= VaR_hedged_dynamic].mean()

# 输出对冲后的总组合结果
print(f"Hedged Portfolio - Mean: ${mean_hedged_dynamic:.2f}")
print(f"Hedged Portfolio - VaR: ${abs(VaR_hedged_dynamic):.2f}")
print(f"Hedged Portfolio - ES: ${abs(ES_hedged_dynamic):.2f}")

# 计算并输出每个单独组合的均值、VaR 和 ES
for portfolio_name, pnl in portfolio_pnl.items():
    mean_individual = pnl.mean()
    VaR_individual = np.percentile(pnl, 5)
    ES_individual = pnl[pnl <= VaR_individual].mean()
    
    print(f"Portfolio {portfolio_name} - Mean: ${mean_individual:.2f}, VaR: ${abs(VaR_individual):.2f}, ES: ${abs(ES_individual):.2f}")


Hedged Portfolio - Mean: $-0.09
Hedged Portfolio - VaR: $21.98
Hedged Portfolio - ES: $27.10
Portfolio Straddle - Mean: $-0.04, VaR: $9.40, ES: $11.60
Portfolio SynLong - Mean: $-0.04, VaR: $10.84, ES: $13.37
Portfolio CallSpread - Mean: $-0.01, VaR: $2.41, ES: $2.97
Portfolio PutSpread - Mean: $0.00, VaR: $0.65, ES: $0.82
Portfolio Call  - Mean: $-0.04, VaR: $10.14, ES: $12.48
Portfolio Put  - Mean: $0.00, VaR: $0.72, ES: $0.91
Portfolio CoveredCall - Mean: $0.04, VaR: $9.16, ES: $11.50
Portfolio ProtectedPut - Mean: $0.00, VaR: $0.25, ES: $0.32


收益对比：对冲后大部分组合的 Mean 接近于 0，说明对冲操作有效地减少了收益的波动性。  
风险对比：对冲后总组合的 VaR 和 ES 均增加，说明对冲操作在极端市场情况下会增加潜在风险。这可能由于对冲操作在市场剧烈波动时效率降低。  
策略选择：CallSpread 和 PutSpread 在对冲后风险显著减少，而 Straddle 和 SynLong 在对冲后极端风险较高。  
Return Comparison: Mean is close to 0 for most portfolios after hedging, indicating that the hedging operation effectively reduces the volatility of returns.  
Risk Comparison: The VaR and ES of the total portfolio increase after hedging, indicating that the hedging operation increases the potential risk in extreme market conditions. This may be due to the fact that hedging operations are less efficient in times of high market volatility.  
Strategy selection: CallSpread and PutSpread have significantly less risk after hedging, while Straddle and SynLong have higher extreme risk after hedging.

# Problem 3
Use the Fama French 3 factor return time series (F-F_Research_Data_Factors_daily.CSV) as well as the Carhart Momentum time series(F-F_Momentum_Factor_daily.CSV) to fit a 4 factor model to the following stocks.  
AAPL META UN MA  
MSFT NVDA HD PFE  
AMZN BRK-B PG XOM  
TSLA JPM V DIS  
GOOGL JNJ BAC CSCO  
Fama stores values as percentages, you will need to divide by 100 (or multiply the stock returns by 100) to get like units.  
Based on the past 10 years of factor returns, find the expected annual return of each stock.  
Construct an annual covariance matrix for the 10 stocks.  
Assume the risk free rate is 0.05. Find the super efficient portfolio

In [ ]:
import yfinance as yf

# 定义股票代码列表
stocks = ["AAPL", "META", "UNH", "MA", "MSFT", "NVDA", "HD", "PFE", 
          "AMZN", "BRK-B", "PG", "XOM", "TSLA", "JPM", "V", "DIS", 
          "GOOGL", "JNJ", "BAC", "CSCO"]

# 下载过去 10 年的每日股票价格
stock_data = yf.download(stocks, start="2013-01-01", end="2023-01-01")['Adj Close']


stock_data.to_csv("/Users/apple/Desktop/stock_prices.csv")


[*********************100%***********************]  20 of 20 completed


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.optimize import minimize

stock_data = pd.read_csv('/Users/apple/Desktop/FT545 - Week7/stock_prices.csv', index_col='Date', parse_dates=True)
momentum_factor_data = pd.read_csv('/Users/apple/Desktop/FT545 - Week7/F-F_Momentum_Factor_daily.CSV', index_col='Date', parse_dates=True)
fama_french_factors_data = pd.read_csv('/Users/apple/Desktop/FT545 - Week7/F-F_Research_Data_Factors_daily.CSV', index_col='Date', parse_dates=True)

In [166]:
# Step 2: 数据清理，确保动量因子和因子数据的列名无空格
momentum_factor_data.columns = momentum_factor_data.columns.str.strip()
fama_french_factors_data.columns = fama_french_factors_data.columns.str.strip()

# 确保单位一致，将所有因子数据除以100
fama_french_factors_data[['Mkt-RF', 'SMB', 'HML', 'RF']] /= 100
momentum_factor_data['Mom'] /= 100

In [164]:
# Step 3: 计算每日收益率
stock_returns = stock_data.pct_change().dropna()  # 转换为收益率

In [165]:
# Step 4: 筛选因子和动量数据在股票数据的日期范围内
start_date, end_date = stock_returns.index.min(), stock_returns.index.max()
fama_french_filtered = fama_french_factors_data[(fama_french_factors_data.index >= start_date) & (fama_french_factors_data.index <= end_date)]
momentum_factor_filtered = momentum_factor_data[(momentum_factor_data.index >= start_date) & (momentum_factor_data.index <= end_date)]

# 输出筛选后的因子数据
print("\n筛选后的Fama-French因子数据：")
print(fama_french_filtered.head())
print("\n筛选后的动量因子数据：")
print(momentum_factor_filtered.head())

# Step 5: 合并因子数据
factors_combined = fama_french_filtered.join(momentum_factor_filtered, how="inner")
print("\n合并后的因子数据：")
print(factors_combined.head())


筛选后的Fama-French因子数据：
                  Mkt-RF           SMB           HML   RF
Date                                                     
2013-01-03 -1.400000e-07  1.100000e-07  4.000000e-08  0.0
2013-01-04  5.500000e-07  1.200000e-07  3.600000e-07  0.0
2013-01-07 -3.100000e-07 -1.000000e-07 -3.500000e-07  0.0
2013-01-08 -2.700000e-07  5.000000e-08  0.000000e+00  0.0
2013-01-09  3.400000e-07  2.700000e-07 -4.000000e-07  0.0

筛选后的动量因子数据：
                     Mom
Date                    
2013-01-03 -4.300000e-07
2013-01-04 -3.700000e-07
2013-01-07  1.900000e-07
2013-01-08  8.400000e-07
2013-01-09  8.000000e-08

合并后的因子数据：
                  Mkt-RF           SMB           HML   RF           Mom
Date                                                                   
2013-01-03 -1.400000e-07  1.100000e-07  4.000000e-08  0.0 -4.300000e-07
2013-01-04  5.500000e-07  1.200000e-07  3.600000e-07  0.0 -3.700000e-07
2013-01-07 -3.100000e-07 -1.000000e-07 -3.500000e-07  0.0  1.900000e-07
2013-01-08 -2

In [163]:
# Step 6: 计算每只股票的因子暴露（回归系数）
factor_exposures = {}
excess_stock_returns = stock_returns.subtract(factors_combined['RF'], axis=0)  # 计算超额收益

In [162]:
for stock in excess_stock_returns.columns:
    y = excess_stock_returns[stock]
    X = factors_combined[['Mkt-RF', 'SMB', 'HML', 'Mom']]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    factor_exposures[stock] = model.params

# 因子暴露数据表
exposure_df = pd.DataFrame(factor_exposures).T

# Step 7: 计算年化预期收益
annual_factor_means = factors_combined[['Mkt-RF', 'SMB', 'HML', 'Mom']].mean() * 252

expected_annual_returns = exposure_df[['Mkt-RF', 'SMB', 'HML', 'Mom']].dot(annual_factor_means) + (exposure_df['const'] * 252)
print("\nThe annualized expected return of each stock：")
print(expected_annual_returns)


The annualized expected return of each stock：
AAPL     0.247294
AMZN     0.240931
BAC      0.164900
BRK-B    0.138633
CSCO     0.147967
DIS      0.097690
GOOGL    0.195104
HD       0.211549
JNJ      0.134421
JPM      0.173053
MA       0.235449
META     0.219277
MSFT     0.272361
NVDA     0.488790
PFE      0.133443
PG       0.124346
TSLA     0.558743
UNH      0.275352
V        0.206561
XOM      0.100009
dtype: float64


In [161]:
# Step 8: 计算年化协方差矩阵
daily_cov_matrix = excess_stock_returns.cov()
annual_cov_matrix = daily_cov_matrix * 252

In [160]:
from scipy.optimize import minimize

# 超级有效组合优化
risk_free_rate = 0.05

# 定义目标函数（负夏普比率）
def neg_sharpe(weights):
    portfolio_return = np.dot(weights, expected_annual_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(annual_cov_matrix, weights)))
    return -(portfolio_return - risk_free_rate) / portfolio_volatility

# 约束条件：权重和为1
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
bounds = [(0, 1) for _ in range(len(expected_annual_returns))]

# 初始权重均匀分配
initial_weights = np.array([1 / len(expected_annual_returns)] * len(expected_annual_returns))

# 优化
result = minimize(neg_sharpe, initial_weights, bounds=bounds, constraints=constraints)
optimal_weights = result.x

# 输出优化结果
print("\nOptimal portfolio weight：")
print(optimal_weights)
print("\nThe expected annualized return of the portfolio：", np.dot(optimal_weights, expected_annual_returns))
print("The annualized volatility of the portfolio：", np.sqrt(np.dot(optimal_weights.T, np.dot(annual_cov_matrix, optimal_weights))))
print("The Sharpe ratio of the portfolio：", -(result.fun))



Optimal portfolio weight：
[0.00000000e+00 0.00000000e+00 4.38098664e-17 6.51312431e-17
 9.71749803e-17 1.74895837e-16 4.74470112e-17 0.00000000e+00
 1.15568686e-16 0.00000000e+00 1.20657149e-17 0.00000000e+00
 7.18810742e-03 2.88208424e-01 2.15524647e-18 9.60536591e-18
 2.08992340e-01 4.95611128e-01 0.00000000e+00 0.00000000e+00]

The expected annualized return of the portfolio： 0.39607178349778605
The annualized volatility of the portfolio： 0.2760571665835868
The Sharpe ratio of the portfolio： 1.2536236163715013


最优组合主要集中在少数几只高预期收益的股票上，如 TSLA、NVDA 和 MSFT，这些股票的高回报与合理的风险调整回报率使其在最优解中获得了显著权重。该组合的年化预期收益为 39.61%，波动率为 27.61%，夏普比率达到 1.25，表明在承担较高风险的前提下，组合实现了较优的风险调整回报。  
The optimal portfolio focuses on a few high expected return stocks such as TSLA, NVDA, and MSFT, whose high returns with reasonable risk-adjusted returns give them significant weight in the optimal solution. The portfolio has an annualized expected return of 39.61%, a volatility of 27.61%, and a Sharpe ratio of 1.25, suggesting that the portfolio achieves superior risk-adjusted returns while taking on a higher level of risk.